# Implementação de um data lakehouse para dados geoespaciais com Eelta Lake

In [2]:
import pyspark
from delta import *
import os

## Configurando Spark

In [3]:
builder = pyspark.sql.SparkSession.builder.appName("MyApp") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

24/08/09 22:59:24 WARN Utils: Your hostname, jimi-Ufscar-Midas resolves to a loopback address: 127.0.1.1; using 200.18.98.172 instead (on interface eno1)
24/08/09 22:59:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/usr/local/lib/python3.10/dist-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jimi/.ivy2/cache
The jars for the packages stored in: /home/jimi/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-98a5bd0f-4a78-437d-a862-1b2bcf66dad3;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.2.0 in central
	found io.delta#delta-storage;3.2.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 102ms :: artifacts dl 5ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.2.0 from central in [default]
	io.delta#delta-storage;3.2.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   | 

# Nível Bronze

In [15]:
csvs = os.listdir('data')

for csv in csvs:
    if not os.path.exists(f'VED-lakehouse/bronze/{csv[4:-9]}/'):
        df = spark.read.format("csv").option("header", True).load(f"./data/{csv}")
        
        columns = []
        for column in df.columns:

            if 'Air' in column:
                column = column.replace('[', '_')[:-1]
            else:
                column = column.split('(')[0].split('[')[0]

            column = column.replace(' ', '_').lower()
            columns.append(column)

        df = df.toDF(*columns)
        
        df.write.format("delta").save(f"./VED-lakehouse/bronze/{csv[4:-9]}")


24/08/09 23:17:47 WARN MemoryManager: Total allocation exceeds 95,00% (950.822.490 bytes) of heap memory
Scaling row group sizes to 88,55% for 8 writers
24/08/09 23:17:47 WARN MemoryManager: Total allocation exceeds 95,00% (950.822.490 bytes) of heap memory
Scaling row group sizes to 78,71% for 9 writers
24/08/09 23:17:47 WARN MemoryManager: Total allocation exceeds 95,00% (950.822.490 bytes) of heap memory
Scaling row group sizes to 70,84% for 10 writers
24/08/09 23:17:47 WARN MemoryManager: Total allocation exceeds 95,00% (950.822.490 bytes) of heap memory
Scaling row group sizes to 64,40% for 11 writers
24/08/09 23:17:47 WARN MemoryManager: Total allocation exceeds 95,00% (950.822.490 bytes) of heap memory
Scaling row group sizes to 59,03% for 12 writers
24/08/09 23:17:47 WARN MemoryManager: Total allocation exceeds 95,00% (950.822.490 bytes) of heap memory
Scaling row group sizes to 64,40% for 11 writers
24/08/09 23:17:48 WARN MemoryManager: Total allocation exceeds 95,00% (950.822

# Nível Prata

In [17]:
df = spark.read.format('delta').option("header", True).load('VED-lakehouse/bronze/171101')
df.show()

24/08/09 23:19:23 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------------+-----+----+---------+-------------+--------------+-------------+-------------+----------+-------------+---+---------+-------------------------+----------------------------+------------+------------------+--------------+------------------+---------------------------+---------------------------+--------------------------+--------------------------+
|       daynum|vehid|trip|timestamp|     latitude|     longitude|vehicle_speed|          maf|engine_rpm|absolute_load|oat|fuel_rate|air_conditioning_power_kw|air_conditioning_power_watts|heater_power|hv_battery_current|hv_battery_soc|hv_battery_voltage|short_term_fuel_trim_bank_1|short_term_fuel_trim_bank_2|long_term_fuel_trim_bank_1|long_term_fuel_trim_bank_2|
+-------------+-----+----+---------+-------------+--------------+-------------+-------------+----------+-------------+---+---------+-------------------------+----------------------------+------------+------------------+--------------+------------------+-------------------